In [3]:
from omegaconf import OmegaConf
import torch
torch.set_grad_enabled(False)
import hydra
import wandb.sdk.data_types.video as wv
from diffusion_policy.env.pusht.pusht_image_env import PushTImageEnv
from diffusion_policy.gym_util.multistep_wrapper import MultiStepWrapper
from torchvision.transforms.functional import to_pil_image
from diffusion_policy.common.pytorch_util import dict_apply
import torch.nn.functional as F
from diffusion_policy.gym_util.video_recording_wrapper import VideoRecordingWrapper, VideoRecorder
import pathlib

def prepare_obs(obs):
    np_obs_dict = {
        'image': F.interpolate(torch.from_numpy(obs['image']), size=(96, 96), mode='bilinear', align_corners=False)[None, ...],
        'origin_image': torch.from_numpy(obs['image']), 
        'agent_pos': torch.from_numpy(obs['agent_pos'][None, ])
    }
    return np_obs_dict

In [4]:
cfg = OmegaConf.load(f'./configs/arp.yaml')
arp_policy = hydra.utils.instantiate(cfg.policy)
arp_policy.load_state_dict(torch.load('./weights/epoch=2000-test_mean_score=0.865.ckpt')['state_dicts']['model'])


============= Initialized Observation Utils with Obs Spec =============

using obs modality: rgb with keys: ['image']
using obs modality: depth with keys: []
using obs modality: scan with keys: []
using obs modality: low_dim with keys: []


/root/miniconda3/envs/arp/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/root/miniconda3/envs/arp/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/tmp/ipykernel_26207/2474287943.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This

<All keys matched successfully>

In [5]:
env = MultiStepWrapper(VideoRecordingWrapper(
                    PushTImageEnv(
                        render_size=256
                    ),
                    video_recoder=VideoRecorder.create_h264(
                        fps=10,
                        codec='h264',
                        input_pix_fmt='rgb24',
                        crf=22,
                        thread_type='FRAME',
                        thread_count=1
                    ),
                    file_path=None,
                    steps_per_render=1
                ),
                n_obs_steps=cfg.policy.n_obs_steps,
                n_action_steps=cfg.policy.n_action_steps,
                max_episode_steps=200)

env.env.video_recoder.stop()
demo_path = pathlib.Path('./outputs/demo')
demo_path.mkdir(parents=True, exist_ok=True)
filename = demo_path.joinpath(
    'media', wv.util.generate_id() + ".mp4")
filename.parent.mkdir(parents=False, exist_ok=True)
filename = str(filename)
env.env.file_path = filename

In [6]:
env.seed(10)
obs = env.reset() # obs['agent_pos'].shape = (2, 2)

arp_policy.eval()
done = False
while not done:
    action_dict = arp_policy.predict_action(prepare_obs(obs))
    obs, reward, done, info = env.step({k: v.detach().to('cpu').numpy() for k, v in action_dict.items()}['action'][0])

obs = env.reset()

"demonstration is saved to ", env.env.file_path

('demonstration is saved to ', 'outputs/demo/media/1ode8w3e.mp4')

In [7]:
from tqdm.auto import trange
for s in trange(1, 101):
    env.seed(s)
    filename = demo_path.joinpath(
    'media',  f"{s}.mp4")
    filename.parent.mkdir(parents=False, exist_ok=True)
    filename = str(filename)
    env.env.file_path = filename

    obs = env.reset() # obs['agent_pos'].shape = (2, 2)

    arp_policy.eval()
    done = False
    while not done:
        action_dict = arp_policy.predict_action(prepare_obs(obs))
        obs, reward, done, info = env.step({k: v.detach().to('cpu').numpy() for k, v in action_dict.items()}['action'][0])

    obs = env.reset()

  0%|          | 0/100 [00:00<?, ?it/s]